<img src="./logo_UTN.svg" align="right" width="150" /> 

#### Teoría de los Circuitos 2 - R4001

# Trabajo práctica de laboratorio 1 : Teoría Moderna y Filtrado activado
#### Lugano Damian - Leg 1756990

Integrantes:

+ Lugano Damian 
+ Nuñez Ana
+ Fiasche Nahuel

--------------------------------------------------
### **Objetivos**

+ Consolidar los conceptos de teoría moderna mediante la implementación circuital.
+ Simular e implementar el filtro con componentes activos de precisión.
+ Medir las partes de la función transferencia para frecuencias menores a 100 kHz.


### **Descripción**

El trabajo práctico consiste en el diseño, análisis, medición y discusión de un filtro activo.

En este caso se propone diseñar el filtro A, que es un filtro pasabajos con función de aproximación Chebyshev. A continuación su plantilla:

<figure>
<center>
<img src="./pics/plantilla.png" width = "800">
<figcaption> Figura 1. Plantilla del filtro A </figcaption>
</figure>

Para implementar la funcion transferencia del filtro diseñado se utilizará el circuito integrado UAF42 ([datasheet](https://www.ti.com/lit/ds/symlink/uaf42.pdf?ts=1653513515595&ref_url=https%253A%252F%252Fwww.ti.com%252Fproduct%252FUAF42%253Futm_source%253Dsupplyframe%2526utm_medium%253DSEP%2526utm_campaign%253Dnot_alldatasheet%2526DCM%253Dyes%2526dclid%253DCOTE4LXH-_cCFaCFlQIdQoELbA)).

#### UAF42 - Universal Active Filter

El CI UAF42 es un filtro activo universal configurado para un amplio rango de filtros pasa bajos, pasa altos y pasa banda. Por otro lado posee capacitores integrados con una tolerancia al 0.5%. Esto permite obtener respuestas en frecuencia del filtro muy parecidas a las simulaciones.

<figure>
<center>
<img src=./pics/UAF42_1.png width = "800">
<figcaption> Figura 2. Circuito interno del UAF42 </figcaption>
</figure>

<figure>
<center>
<img src=./pics/UAF42_2.png width = "500">
<figcaption> Figura 3. PINOUT del UAF42 </figcaption>
</figure>


## **Análisis**

#### Plantilla

Chebyshev : 
+ $f_c01.5KHz$
+ $f_s=4.5KHz$
+ $\alpha_{MAX} = 0.5dB$
+ $\alpha_{MIN}=16dB$

#### 1 - Obtener el orden del filtro y el $\epsilon$

Primero se establece la norma de frecuencia :

\begin{equation}
    \Omega_\omega=2 \pi f_c \hspace{1cm} \Longrightarrow \hspace{1cm} \omega_c=1 \hspace{0.5cm} y \hspace{0.5cm} \omega_s=3
\end{equation}

\begin{equation}
    \epsilon²=10^{\frac{\alpha_{max}}{10}}-1 = 0.122 \hspace{1cm} \Longrightarrow \epsilon=0.349
\end{equation}

\begin{equation}
    \alpha_{min}=10 * log[ 1 + \epsilon² * cosh²(n * cosh^{-1}(\omega_s)) ]
\end{equation}

Se itera: 

+ $\alpha_{min2} = 15.59 dB$
+ $\alpha_{min2} = 30.77 dB$

#### Por lo tanto se elije orden n=3

La transferencia de un cheby está caracterizada por:


\begin{equation}
    | T(S) . T(-S) | = | T(j\omega) |² = \frac{1}{1+\epsilon² * C_3(\omega)²}
\end{equation}

Donde el polinomio de Cheby para orden 3 es $C_3(\omega)=2\omega*C_2 - C_1 = 4\omega³ -2\omega$

Para evitar hacer el desarrollo matemática que conlleva el diseño de un chebyshev. Se utilizó el simulador numérico en python


In [1]:
#----------------- imports ---------------------
from scipy.signal import TransferFunction
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as ss

from pytc2.sistemas_lineales import analyze_sys, pretty_print_bicuad_omegayq, tf2sos_analog, pretty_print_SOS

from pytc2.general import print_subtitle

In [2]:
#----------------------Definicion de las variables---------------------

xi = 0.349
n = 3
alpha_max = 0.5

In [3]:
#---------------------------Definicion de H-------------------------

[z, p, k] = ss.cheb1ap(n, alpha_max)

[num, den] = ss.zpk2tf(z, p, k)

sos_lp = tf2sos_analog(num, den)
sos_lp[sos_lp < 1e-6] = 0.0

#-----------------------------Visualizacion----------------------------

pretty_print_SOS(sos_lp, mode='omegayq')
print("z = ", z, "\n", "p = ", p, "\n", "k =", k)

<IPython.core.display.Math object>

z =  [] 
 p =  [-0.31322824+1.02192749j -0.62645649-0.j         -0.31322824-1.02192749j] 
 k = 0.7156937903107974


Como se puede ver se llegó a la función transferencia :

\begin{equation}
    T(S)=\frac{\omega_{02} }{s + \omega_{02} } . \frac{\omega_{01}² }{s^2 + s \frac{\omega_{01}}{Q} + \omega_{01}^2} =\frac{0.6264 }{s + 0.6265 } . \frac{1.143 }{s^2 + s \frac{1.069}{1.706} + 1.069^2} 
\end{equation}

Donde se puede ver en el sistema de segundo orden (SOS, por sus siglas en ingles) que el valor de $Q=1.71$ y que $\omega_{01}=1.069\approx1$ y por otro lado $\omega_{02}=0.626$

### **Sintesis del filtro**

Para la primer etapa, es decir la SOS, se utilizará el CI UAF42 y para la segunda etapa de orden 1 se implementará un circuito RC. Por último se utilizará una etapa seguidora a la salida del filtro para evitar que la impedancia de las puntas de osciloscopio se ponga en paralelo al capacitor de salida y modifique los parámetros del mismo.

<figure>
<center>
<img src=./pics/blockdiagram.png width = "800">
<figcaption> Figura 4. Diagrama en bloques del filtro </figcaption>
</figure>

#### Sintesis de los valores de los componentes externos del circuito UAF42

Se utilizaron las ecuaciones de diseño provistas por la hoja de datos para par de polos inversores, a partir de la siguiente configuración de componentes:

<figure>
<center>
<img src=./pics/design_eq.png width = "800">
<figcaption> Figura 5. Ecuaciones de diseño para par de polos inversores </figcaption>
</figure>

A partir de dichas ecuaciones, se desarrollo un sistema de ecuaciones para obtener los valores de los componentes externos, es decir RG, RQ y el par RF1 RF2.

Primero se debe tener en cuenta que $R_1=R_2=R_4=50K$ y que $C_1=C_2=1000pF$. Por lo tanto:

+ $\omega_{01}²=\frac{1}{R_{F1}.R_{F2}.C²}=1²$

+ $Q=(1+\frac{R_4}{R_G}) \frac{1}{\frac{1}{R_1}+\frac{1}{R_2}+\frac{1}{R_G}} (\frac{1}{R_1 . R_2})^{1/2} = 1.706$

+ $A_{LP}=\frac{R_1}{R_G}=1$

Sin embargo para obtener el valor de los componetes desnormalizados, se deberá desnormalizar la transferencia primero.

\begin{equation}
    T1(S)=\frac{1.143 }{s^2 + s \frac{1.069}{1.706} + 1.069^2} \hspace{0.5cm} \Longrightarrow \hspace{0.5cm} \frac{1.143 }{\frac{s^2}{\Omega\omega²} + \frac{s}{\Omega\omega} \frac{1.069}{1.706} + 1.069^2}
\end{equation}

\begin{equation}
    T1(S)=\frac{1.143  \Omega\omega² }{s^2 + s \hspace{0.1cm} \Omega\omega \hspace{0.1cm} \frac{1.069}{1.706} + 1.069^2 \hspace{0.1cm} \Omega\omega²}
\end{equation}

Recordando que $\Omega\omega=\omega_p=2 \pi f_p \hspace{0.5cm} \Longrightarrow \hspace{0.5cm} \Omega\omega=3000\pi$

\begin{equation}
    T1(S)=\frac{ 101.5\times10⁶ }{s^2 + s \frac{10.07\times10³}{1.706} + 101.5\times10⁶}
\end{equation}

A partir de esto. Ahora:

\begin{equation}
\omega_{01}²=\frac{1}{R_{F1}.R_{F2}.10^{-18}}=\frac{10^{18}}{R_{F1}.R_{F2}} \hspace{0.5cm} \Longrightarrow \hspace{0.5cm} R_{F1}.R_{F2}=\frac{10^{18}}{\omega_o²}=R_{F1}.R_{F2}=\frac{10^{18}}{101.5\times10⁶} \hspace{0.5cm} \Longrightarrow \hspace{0.5cm} R_{F1}.R_{F2} = 9.852\times10⁹
\end{equation}

\begin{equation}
Adoptando \hspace{0.5cm} \rightarrow  R_{F1}=R_{F2} \hspace{0.5cm} \Longrightarrow \hspace{0.5cm} R_{F1}=R_{F2}\approx99.25K\Omega 
\end{equation}

Por otro lado, la ganancia:

\begin{equation}
A_{LP}=\frac{R_1}{R_G}=1 \hspace{0.5cm} \Longrightarrow \hspace{0.5cm} R_G=R_1=50K\Omega
\end{equation}

Por último el Q:

\begin{equation}
Q=(1+\frac{R_4}{R_G}) \frac{50K\Omega}{3} (\frac{1}{(50K\Omega)²})^{1/2} = (1+\frac{R_4}{R_G}) \times \frac{1}{3} \hspace{0.5cm} \Longrightarrow \hspace{0.5cm} R_G=12.14K\Omega
\end{equation}

Por lo que finalmente, pasando todos los valores de componentes a valores comerciales queda:

+ $R_{F1}=R_{F2}=100K\Omega$
+ $R_G=50K\Omega \hspace{0.5cm} \rightarrow$ se puede hacer con 2 resistencias de 100K en paralelo
+ $R_Q=12K\Omega$

#### Filtro RC

Para sintetizar la transferencia de primero orden se utilizaró una resistencia con un capacitor en derivacion.

<figure>
<center>
<img src=./pics/filtroRC.png width = "500">
<figcaption> Figura 6. Circuito filtro RC </figcaption>
</figure>

Su transferencia está caracterizada de la siguiente manera:

\begin{equation}
    T(S)=\frac{\omega_o}{s+\omega_o}=\frac{\frac{1}{R_x.C_x}}{s+\frac{1}{R_x.C_x}}=\frac{0.6264 }{s + 0.6265 }
\end{equation}

Para obtener el valor de los componentes, se tomó como norma de impedancia al capacitor, de manera de poder proponer un valor comercial de capacitancia y luego, en base al valor de resistencia resultante, ver de conseguir un valor comercial cercano o utilizar un trimpot.

Entonces $\hspace{0.5cm} \Omega_Z=C_X \hspace{0.5cm} \Longrightarrow \hspace{0.5cm} C=1$

De esta manera $ \hspace{0.5cm} T(s)=\frac{\frac{1}{R_X}}{s+\frac{1}{R_X}}  \hspace{0.5cm} \Longrightarrow \hspace{0.5cm} \frac{1}{R_X}=0.626 \hspace{0.5cm} \Longrightarrow \hspace{0.5cm} R_X=1.6$

Por último, se propone como valor de capacitor $C_x=10nF$. Por lo tanto:

$C=\frac{C'}{\Omega_z . \Omega_\omega}=10nF$

$\Omega_z=\frac{C'}{C . \Omega_\omega} = \frac{1}{10nF . 3000 \pi} = 10.61K\Omega$

$R_x=R_x' . \Omega_Z=16.976K\Omega$

Por lo tanto se adoptará $R_x=17K\Omega$


#### Lista de componentes

